# Everplans Data Engineer Code Test

#### The data provided is some basic activities done by users like creating an account, activating, logging in, loggin out and add contents to vaults provided.

### The primary analysis of the data includes checking for missing,partial or invalid data
- Importing the required packages and reading the dataset provided into pandas data frame "user_activity"
- Asuming the data set is in the same folder as the jupyter notebook
- printing the first few records

In [1]:
import pandas as pd
import numpy as np
user_activity = pd.read_csv("raw_event_log.csv")
user_activity.head()

,user_id,timestamp,event
0,eeb5138977d2e18f2f9071624e4d5757,2020-01-01 00:01:22,create
1,e28be90ec2538ff7708ee649a4ee8d61,2020-01-01 00:04:25,create
2,c784798b5d179ca695a405f2c737797b,2020-01-01 00:23:35,create
3,35ec8d997e035d9a7ca103654661f4d9,2020-01-01 00:25:48,create
4,070292c8307f41c18e02bb8cd09cfff3,2020-01-01 00:28:17,create


Checking for datatypes to decide if conversion are required

In [2]:
user_activity.dtypes

user_id      object
timestamp    object
event        object
dtype: object

Converting the timestamp column into datetime format for further processing

In [3]:
from datetime import datetime as dt
user_activity['timestamp'] = pd.to_datetime(user_activity['timestamp'])
user_activity.dtypes

user_id              object
timestamp    datetime64[ns]
event                object
dtype: object

Checking if there are any null values in total and also getting count of null values if any in the column wise

In [4]:
user_activity.isnull().values.any(),user_activity.isnull().sum()

(False,
 user_id      0
 timestamp    0
 event        0
 dtype: int64)

Check for valid events, to clean up if any misspelled or worng events

In [5]:
user_activity.event.unique()

array(['create', 'activate', 'add_to_vault', 'logout', 'login'],
      dtype=object)

Checking for number of unique users

In [6]:
user_activity.user_id.nunique()

12001

Checking if the user_id column has only alpha numeric characters

In [7]:
user_activity['user_id'].map(lambda x:x.isalnum()).unique()

array([ True])

# Question 1

### create a dataset that has 1 record for each unique user ID, with the following dimensions and measures:
- Date of activation. (user may or may not have activated).
- total_logins: The total number of times the user has logged in.
- total_vault_events: The total number of times a user has done the add_to_vault event.
- time_to_activate: The number of seconds it took for teh user to go from create to activate.



### Choosing only activated users

- As mentioned not all users will have activated their account.
- Assuming activation of account is required to do any other activity.

In [8]:
activated_users = user_activity[user_activity['event']=='activate'].reset_index(drop=True)
activated_users.head()

,user_id,timestamp,event
0,c16805f31723c4628121655eb3b6b9b8,2020-01-01 00:57:06,activate
1,eeb5138977d2e18f2f9071624e4d5757,2020-01-01 01:22:02,activate
2,6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01 01:36:42,activate
3,5992c483bdab1c36edb7e9ceaea01af0,2020-01-01 03:36:35,activate
4,4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01 04:19:43,activate


* Calculating the total number of times a user logged in into the account

* Using group by fuction to get the count of logins

In [9]:
grouped_users= user_activity.groupby(['user_id','event']).count().reset_index()
loggedin_users = grouped_users[grouped_users['event']=='login']

In [10]:
#Checking for datatypes the count is taken as a floating value
loggedin_users.dtypes

user_id      object
event        object
timestamp     int64
dtype: object

- Utilizing the join method to join the two dataframes for activated users and loggedin users based on the "user_id" column.
- Applying a left join so as to get the login counts for activated user alone

In [11]:
activated_users=activated_users.set_index('user_id').join(loggedin_users.set_index('user_id'),how='left', rsuffix='_login',on= 'user_id')

- Inserting 0s where the user might have activated the account bot might not have logged in.
- The join would result in NaN values for users who have not loggedin into the account

In [12]:
activated_users['timestamp_login'] = activated_users['timestamp_login'].fillna(0)
activated_users.head()

,timestamp,event,event_login,timestamp_login
user_id,,,,
c16805f31723c4628121655eb3b6b9b8,2020-01-01 00:57:06,activate,login,7.0
eeb5138977d2e18f2f9071624e4d5757,2020-01-01 01:22:02,activate,NaN,0.0
6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01 01:36:42,activate,login,3.0
5992c483bdab1c36edb7e9ceaea01af0,2020-01-01 03:36:35,activate,login,5.0
4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01 04:19:43,activate,login,8.0


__Dropping unwanted columns and renaming the columns__

In [13]:
activated_users=activated_users.drop(['event','event_login'],axis=1)
activated_users = activated_users.rename(columns={"timestamp": "date_of_activation", "timestamp_login": "login_count"})

In [14]:
activated_users.head()

,date_of_activation,login_count
user_id,,
c16805f31723c4628121655eb3b6b9b8,2020-01-01 00:57:06,7.0
eeb5138977d2e18f2f9071624e4d5757,2020-01-01 01:22:02,0.0
6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01 01:36:42,3.0
5992c483bdab1c36edb7e9ceaea01af0,2020-01-01 03:36:35,5.0
4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01 04:19:43,8.0


In [15]:
vault_activity_counts = grouped_users[grouped_users['event']=='add_to_vault']

**Calculating the total vault events and joining with the already activated users**

In [16]:
activated_users=activated_users.join(vault_activity_counts.set_index('user_id'),how='left', rsuffix='_vault_events',on= 'user_id')

In [17]:
activated_users.head()

,date_of_activation,login_count,event,timestamp
user_id,,,,
c16805f31723c4628121655eb3b6b9b8,2020-01-01 00:57:06,7.0,add_to_vault,16.0
eeb5138977d2e18f2f9071624e4d5757,2020-01-01 01:22:02,0.0,add_to_vault,1.0
6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01 01:36:42,3.0,add_to_vault,10.0
5992c483bdab1c36edb7e9ceaea01af0,2020-01-01 03:36:35,5.0,add_to_vault,16.0
4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01 04:19:43,8.0,add_to_vault,27.0


In [18]:
#Dropping the unwanted columns
activated_users=activated_users.drop(['event'],axis=1)
activated_users = activated_users.rename(columns={ "timestamp": "total_vault_events"})

In [19]:
activated_users.head()

,date_of_activation,login_count,total_vault_events
user_id,,,
c16805f31723c4628121655eb3b6b9b8,2020-01-01 00:57:06,7.0,16.0
eeb5138977d2e18f2f9071624e4d5757,2020-01-01 01:22:02,0.0,1.0
6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01 01:36:42,3.0,10.0
5992c483bdab1c36edb7e9ceaea01af0,2020-01-01 03:36:35,5.0,16.0
4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01 04:19:43,8.0,27.0


#### Calculating the time taken for users to activate the accunt after creation
- getting the timestamp when the count of the user was created

In [20]:
create = user_activity[user_activity['event']=='create']
create.head()

,user_id,timestamp,event
0,eeb5138977d2e18f2f9071624e4d5757,2020-01-01 00:01:22,create
1,e28be90ec2538ff7708ee649a4ee8d61,2020-01-01 00:04:25,create
2,c784798b5d179ca695a405f2c737797b,2020-01-01 00:23:35,create
3,35ec8d997e035d9a7ca103654661f4d9,2020-01-01 00:25:48,create
4,070292c8307f41c18e02bb8cd09cfff3,2020-01-01 00:28:17,create


In [21]:
activated_users = create.set_index("user_id").join(activated_users,how='right', rsuffix='_activated',on= 'user_id')

In [22]:
activated_users.head()

,timestamp,event,date_of_activation,login_count,total_vault_events
user_id,,,,,
c16805f31723c4628121655eb3b6b9b8,2020-01-01 00:52:33,create,2020-01-01 00:57:06,7.0,16.0
eeb5138977d2e18f2f9071624e4d5757,2020-01-01 00:01:22,create,2020-01-01 01:22:02,0.0,1.0
6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01 00:57:36,create,2020-01-01 01:36:42,3.0,10.0
5992c483bdab1c36edb7e9ceaea01af0,2020-01-01 02:37:58,create,2020-01-01 03:36:35,5.0,16.0
4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01 03:48:36,create,2020-01-01 04:19:43,8.0,27.0


Calculating the time take to activate the account and converting it to seconds

In [23]:
activated_users['time_to_activate']=(activated_users['date_of_activation']-activated_users['timestamp'])
activated_users['time_to_activate'] =activated_users['time_to_activate'].apply(lambda x:x.total_seconds())

In [24]:
activated_users= activated_users.drop(['timestamp','event'],axis=1)
activated_users.head()

,date_of_activation,login_count,total_vault_events,time_to_activate
user_id,,,,
c16805f31723c4628121655eb3b6b9b8,2020-01-01 00:57:06,7.0,16.0,273.0
eeb5138977d2e18f2f9071624e4d5757,2020-01-01 01:22:02,0.0,1.0,4840.0
6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01 01:36:42,3.0,10.0,2346.0
5992c483bdab1c36edb7e9ceaea01af0,2020-01-01 03:36:35,5.0,16.0,3517.0
4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01 04:19:43,8.0,27.0,1867.0


__Converting the date_of activation column to_date__

In [25]:
activated_users['date_of_activation'] = activated_users['date_of_activation'].dt.date

In [26]:
activated_users.head()

,date_of_activation,login_count,total_vault_events,time_to_activate
user_id,,,,
c16805f31723c4628121655eb3b6b9b8,2020-01-01,7.0,16.0,273.0
eeb5138977d2e18f2f9071624e4d5757,2020-01-01,0.0,1.0,4840.0
6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01,3.0,10.0,2346.0
5992c483bdab1c36edb7e9ceaea01af0,2020-01-01,5.0,16.0,3517.0
4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01,8.0,27.0,1867.0


## Writing to csv

In [27]:
activated_users.to_csv('Activated_user_details.csv',index=True)

# Question 2

### create a dataset that aggreates by month (use activation month) based on the previous dataset, which provides the following dimensions and measures:

- activation_month (can be any reasonable format, string or date)
- average_days_to_activate: Mean of time_to_activate stated in days(use of decimal places is fine)
- average_logins: Mean of total_logins



Calculating the month and year combination for processing month wise data

In [28]:
activated_users['month']=activated_users['date_of_activation'].apply(lambda x:x.strftime('%B-%Y') if not pd.isnull(x) else '')

activated_users.head()

,date_of_activation,login_count,total_vault_events,time_to_activate,month
user_id,,,,,
c16805f31723c4628121655eb3b6b9b8,2020-01-01,7.0,16.0,273.0,January-2020
eeb5138977d2e18f2f9071624e4d5757,2020-01-01,0.0,1.0,4840.0,January-2020
6022e854c088ffa1f2a04dfad4d05e3f,2020-01-01,3.0,10.0,2346.0,January-2020
5992c483bdab1c36edb7e9ceaea01af0,2020-01-01,5.0,16.0,3517.0,January-2020
4c8fd2ea8b8897f45219c9d5a4e3b11c,2020-01-01,8.0,27.0,1867.0,January-2020


__Calculating the time take to activate in days__

In [29]:
activated_users['time_to_activate_days'] =activated_users['time_to_activate'].apply(lambda x:x/86400)

__Calculating the average number of days users take to activate month wise__

In [30]:
month_wise=pd.DataFrame(activated_users.groupby('month')['time_to_activate_days'].mean())
month_wise

,time_to_activate_days
month,
April-2020,0.255664
February-2020,0.250805
January-2020,0.246692
March-2020,0.248384


In [31]:
logins_per_month= pd.DataFrame(activated_users.groupby('month')['login_count'].mean())

__Joining the data to get avg logins per activation month__

In [32]:
month_wise=logins_per_month.join(month_wise,on= 'month')

In [33]:
month_wise = month_wise.rename(columns={ "month": "activation_month",'time_to_activate_days':'average_days_to_activate','login_count':'average_logins'})
month_wise

,average_logins,average_days_to_activate
month,,
April-2020,5.040330,0.255664
February-2020,5.045391,0.250805
January-2020,4.921790,0.246692
March-2020,5.124150,0.248384


In [34]:
month_wise.to_csv('Month_wise_data.csv',index=True)